# **League of legends API**
 
Documentation: https://developer.riotgames.com/apis

# Dependencies

This notebook requires the following libraries:

- `numpy==2.0.0`
- `pandas==2.2.2`
- `matplotlib==3.7.0`
- `snowflake-connector-python==3.11.0`
- `snowflake-sqlalchemy==1.6.1`
- `SQLAlchemy==2.0.31`

You can install them using the `requirements.txt` file.

## Importing Required Libraries
In this section, we import the necessary libraries and assign the API Key that was previously stored in a credentials file. The API Key is a unique identifier used to authenticate requests to the API and ensure that only authorized users have access to its services. 

First, we import the following libraries:

- `requests` for making HTTP requests.
- `pandas` for data manipulation and analysis.
- `snowflake.connector` for connecting to Snowflake.
- `sqlalchemy` for SQL toolkit and Object-Relational Mapping (ORM).
- `credentials` for accessing the stored API Key.
- `datetime` for getting the date time

In [1]:
import requests
import pandas as pd
import snowflake.connector
from sqlalchemy import create_engine, text
from datetime import datetime
import credentials

## API Key Assignment
In this section, we assign the API Key that was previously stored in a credentials file. We use the `credentials` variable to access the API Key and then assign it to the `api_key`  variable.

In [2]:
api_key = credentials.api_key

## **Account**

### Constructing the API Endpoint and Making a Request

In this section, we construct the API endpoint URL to retrieve account information based on the Riot ID and tag line. We then make a request to the Riot Games API and process the response. Our main objective is to get the `puuid`

In [3]:
# Define Variables
tag_Line = 'RFRMA'
game_Name = 'Styles'
# Construct the Endpoint:
server = 'americas.api.riotgames.com'
endpoint = f'https://{server}/riot/account/v1/accounts/by-riot-id/{game_Name}/{tag_Line}?api_key={api_key}'
# Make the Request:
res = requests.get(endpoint).json()
puuid = res['puuid']
df = pd.DataFrame([res])
df


,puuid,gameName,tagLine
0,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,Styles,RFRMA


Using the `puuid` we can obtain our objective's `summoner_id` and therefore more data as `summoner level`

In [4]:
# We assign the latin america north region and make the request
region = 'la1'
endpoint = f'https://{region}.api.riotgames.com/lol/summoner/v4/summoners/by-puuid/{puuid}?api_key={api_key}'
res = requests.get(endpoint).json()
df_summoner = pd.DataFrame([res])
df_summoner

,id,accountId,puuid,profileIconId,revisionDate,summonerLevel
0,5QDTqVFS66ywwKzRVgBxnrZLFL3IbIMhndHAYWaGIgA3eEE,IosCQt_vyGYT5QGNSQasm-pP24wQPww9xv96DiXkkyOAdm4,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,5922,1721795962000,703


### Concatenating DataFrames and Removing Duplicate Columns
We can concat both dataframe into one and drop duplicated rows, specifically on this ocassion `puuid`

In [5]:
df_summoner = pd.concat([df, df_summoner] ,axis=1)
df_summoner = df_summoner.loc[:,~df_summoner.columns.duplicated()]
df_summoner

,puuid,gameName,tagLine,id,accountId,profileIconId,revisionDate,summonerLevel
0,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,Styles,RFRMA,5QDTqVFS66ywwKzRVgBxnrZLFL3IbIMhndHAYWaGIgA3eEE,IosCQt_vyGYT5QGNSQasm-pP24wQPww9xv96DiXkkyOAdm4,5922,1721795962000,703


## **Ranking Mastery**

We're able to use the previous data to obtain the summoner_id and get information about summoner's rank

In [6]:
# We extract the summoner_id from the "df_summoner" DataFrame.
summoner_id = df_summoner["id"][0]
endpoint = f'https://la1.api.riotgames.com/lol/league/v4/entries/by-summoner/{summoner_id}?api_key={api_key}'
summoner_rank = requests.get(endpoint).json()
df_rank = pd.DataFrame([summoner_rank[0]])
df_rank

,leagueId,queueType,tier,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,1ea4bf36-0ec5-4d12-ba7b-9a2f0a9e7ca2,RANKED_SOLO_5x5,EMERALD,II,5QDTqVFS66ywwKzRVgBxnrZLFL3IbIMhndHAYWaGIgA3eEE,22,19,19,False,False,False,False


Now we concat both dataframes

In [7]:
df_summoner = pd.concat([df_summoner, df_rank] ,axis=1)
df_summoner

,puuid,gameName,tagLine,id,accountId,profileIconId,revisionDate,summonerLevel,leagueId,queueType,tier,rank,summonerId,leaguePoints,wins,losses,veteran,inactive,freshBlood,hotStreak
0,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,Styles,RFRMA,5QDTqVFS66ywwKzRVgBxnrZLFL3IbIMhndHAYWaGIgA3eEE,IosCQt_vyGYT5QGNSQasm-pP24wQPww9xv96DiXkkyOAdm4,5922,1721795962000,703,1ea4bf36-0ec5-4d12-ba7b-9a2f0a9e7ca2,RANKED_SOLO_5x5,EMERALD,II,5QDTqVFS66ywwKzRVgBxnrZLFL3IbIMhndHAYWaGIgA3eEE,22,19,19,False,False,False,False


We've got duplicated rows again, so let's drop them

In [8]:
df_summoner.dtypes

puuid            object
gameName         object
tagLine          object
id               object
accountId        object
profileIconId     int64
revisionDate      int64
summonerLevel     int64
leagueId         object
queueType        object
tier             object
rank             object
summonerId       object
leaguePoints      int64
wins              int64
losses            int64
veteran            bool
inactive           bool
freshBlood         bool
hotStreak          bool
dtype: object

In [9]:
# dropping
df_summoner = df_summoner.drop('summonerId', axis=1)

Lets transform `revisionDate` from epoch to datetime

In [10]:
df_summoner['revisionDate']

0    1721795962000
Name: revisionDate, dtype: int64

In [12]:
# transforming
df_summoner['revisionDate'] = pd.to_datetime(df_summoner['revisionDate'],unit='ms')
df_summoner['revisionDate']

0   2024-07-24 04:39:22
Name: revisionDate, dtype: datetime64[ns]

Save the data into a csv file (**Optional**)


In [13]:
df_summoner.to_csv(f'data/summoner_{datetime.today().strftime("%Y_%m_%d")}.csv', index=False)

## **Champion mastery**

### Retrieving Champion Mastery
In this section, we construct an API endpoint URL to retrieve the champion mastery information for a specific player based on their PUUID (Player's Universally Unique Identifier). We then make a request to the Riot Games API to fetch the champion mastery data.

In [14]:
# We extract the PUUID from the "df_summoner" DataFrame.
summ_puuid = df_summoner.loc[0,'puuid']
endpoint = f'https://{region}.api.riotgames.com/lol/champion-mastery/v4/champion-masteries/by-puuid/{summ_puuid}/top?api_key={api_key}'
res = requests.get(endpoint).json()
df_champs_mastery = pd.DataFrame(res)
df_champs_mastery['lastPlayTime'] = pd.to_datetime(df_champs_mastery['lastPlayTime'],unit='ms')
df_champs_mastery

,puuid,championId,championLevel,championPoints,lastPlayTime,championPointsSinceLastLevel,championPointsUntilNextLevel,markRequiredForNextLevel,tokensEarned,championSeasonMilestone,nextSeasonMilestone
0,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,245,36,404598,2023-09-10 15:44:19,42998,-31998,2,0,0,"{'requireGradeCounts': {'B-': 1, 'C-': 4}, 're..."
1,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,55,24,278059,2024-04-02 02:43:36,48459,-37459,2,0,0,"{'requireGradeCounts': {'B-': 1, 'C-': 4}, 're..."
2,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,141,22,259593,2024-01-12 03:41:43,51993,-40993,2,0,0,"{'requireGradeCounts': {'B-': 1, 'C-': 4}, 're..."


## **Match history**

### Retrieving Last 20 Matches
In this section, we construct an API endpoint URL to retrieve the last 20 match IDs for our specific player based on their PUUID. We then make a request to the Riot Games API to fetch the match IDs.

1. **Extract the PUUID:**
   We extract the PUUID from the `df_summoner` DataFrame we got previously.

2. **Define the Match Server:**
   We specify the match server URL.

3. **Construct the Endpoint:**
   We use an f-string to format the endpoint URL, incorporating the `match_server`, `summ_puuid`, and `api_key`.

4. **Make the Request:**
   Using the `requests` library, we send a GET request to the constructed endpoint and parse the JSON response to get the last 20 match IDs.

5. **Store the Match IDs:**
   We store the retrieved match IDs in the `last_20_matches` variable.

In [45]:
# We extract the PUUID from the `df_summoner` DataFrame.
summ_puuid = df_summoner['puuid'][0]
# Define the Match Server
match_server='https://americas.api.riotgames.com'
endpoint = f'{match_server}/lol/match/v5/matches/by-puuid/{summ_puuid}/ids?api_key={api_key}'
res = requests.get(endpoint).json()
last_20_matches = res
last_20_matches

['LA1_1538281487',
 'LA1_1538259341',
 'LA1_1538246510',
 'LA1_1538236591',
 'LA1_1536945190',
 'LA1_1536930761',
 'LA1_1536919278',
 'LA1_1536905667',
 'LA1_1536897034',
 'LA1_1536887147',
 'LA1_1528441003',
 'LA1_1527244320',
 'LA1_1527235181',
 'LA1_1527211799',
 'LA1_1527197782',
 'LA1_1527125837',
 'LA1_1527112416',
 'LA1_1525980196',
 'LA1_1525979364',
 'LA1_1525971622']

### Analyzing a Game Match to Obtain Relevant Columns
In this section, we analyze a specific game match to extract and summarize the important data, we focus on retrieving relevant columns using lists (`basics`,`kills`,`objectives`,`wards`,`pings` and `misc` into **`stats`**).

In [46]:
match_id = res[19]
endpoint = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
res = requests.get(endpoint).json()

In [47]:
basics = [
    'puuid',
    'riotIdGameName',
    'win',
    'lane',
    'role',
    'teamPosition',
    'kills',
    'deaths',
    'assists',
    'totalMinionsKilled',
    'eligibleForProgression',
    'timePlayed',
]
kills = [
    'killingSprees',
    'firstBloodKill',
    'firstTowerKill',
    'doubleKills',
    'tripleKills',
    'quadraKills',
    'pentaKills',
]
wards = [
    'wardsKilled',
    'wardsPlaced',
    'visionWardsBoughtInGame',
    'detectorWardsPlaced',
]
objectives = [
    'objectivesStolen',
    'turretKills',
    'dragonKills',
]
pings = [
    'enemyMissingPings'
]
misc = [
    'item0','item1','item2','item3','item4','item5','item6',
    'goldEarned',
    'largestKillingSpree',
    'largestMultiKill',
    'magicDamageDealtToChampions',
    'magicDamageTaken',
    'neutralMinionsKilled',
    'participantId',
    'physicalDamageDealtToChampions',
    'physicalDamageTaken',
    'teamId',
    'totalDamageDealtToChampions',
    'totalDamageTaken',
    'totalHeal',
    'totalHealsOnTeammates',
    'trueDamageDealtToChampions',
    'trueDamageTaken',
    'visionScore',
]
stats = basics + kills + wards + objectives + pings + misc

### Creating a DataFrame with Relevant Match Statistics
In this section, we create a DataFrame from the match data and focus on the relevant statistics by selecting specific columns.

**1- Define Relevant Columns:**
    - We have defined six lists of relevant columns: `basics`, `kills`,`objectives`,`wards`,`pings` and `misc`. We combine these lists into a single list called `stats` to capture all the necessary statistics.

**2- Create the DataFrame:**
    - We create a DataFrame from the 'participants' data

In [48]:
df = pd.DataFrame(res['info']['participants'])
df_match = df[stats]
df_match

,puuid,riotIdGameName,win,lane,role,teamPosition,kills,deaths,assists,totalMinionsKilled,...,physicalDamageDealtToChampions,physicalDamageTaken,teamId,totalDamageDealtToChampions,totalDamageTaken,totalHeal,totalHealsOnTeammates,trueDamageDealtToChampions,trueDamageTaken,visionScore
0,ya5pvlLlkQXeX_uCHhZb9gYgMjEK33_IoNj6EIXIzbAScX...,FULL K,False,TOP,SOLO,TOP,7,8,2,109,...,16973,16637,100,19194,21365,2777,0,2221,1038,13
1,T41NPvRvPC9ZpMI8nNrNNqKamPNcHfADFU4wUp1UAy6ZbY...,Jyn000,False,JUNGLE,NONE,JUNGLE,3,7,6,11,...,659,21525,100,11125,28159,15588,0,2024,417,17
2,D3kqqIiyJod-cfS1RADHFMhvvCgwh3cFpQlvf9vf4oKGcd...,El GefeMaestro,False,MIDDLE,DUO,MIDDLE,4,8,1,115,...,7734,9712,100,8870,16711,1821,0,420,526,11
3,G_KRm5Udd4rSZCwKAUMrUdO-duYkAdZlTwtQYrQ5CyNThk...,relick96,False,MIDDLE,DUO,BOTTOM,1,5,3,131,...,6269,8997,100,6359,12702,2176,0,66,436,12
4,xFrH49qG59k06fq9NXU_4-7CpI6EzXpkqZzH9GvrsfSNwa...,KenshinGG,False,MIDDLE,SUPPORT,UTILITY,1,5,6,25,...,775,9328,100,5418,13037,508,132,794,657,40
5,TiijSm2W1R384TpxWzchmzpl6G53NYLmqGDlhOM34WnX97...,XxXRaphaXxX,True,TOP,SOLO,TOP,8,3,2,148,...,14450,18584,200,15849,22239,10458,0,618,1043,13
6,IB0QZXqG7WRe4hI9yquSyyxTog7l_1aE-ETUdU6AY6UZYu...,el codazo,True,JUNGLE,NONE,JUNGLE,13,6,6,15,...,15930,14622,200,19511,21784,8805,0,1114,2116,21
7,zRSETMrkt5cmiwxk_6Q14WQgF5w9E0gDA4jCYDXwfcCd01...,Styles,True,MIDDLE,SOLO,MIDDLE,7,6,7,119,...,1146,7522,200,15362,10183,361,0,0,622,8
8,vrgYvZdMPoGyycMWkKJcKK8IxoDhsOrCH7ZaZHM5TeLj7h...,XzJoSmElLxZ,True,BOTTOM,CARRY,BOTTOM,3,0,11,161,...,12974,5967,200,14147,9496,5185,219,828,773,15
9,5YeK5c17ZN4eRkHxUqzV-_3aYCh17HKisEGQO1rlTl_g3-...,ZlTito,True,BOTTOM,SUPPORT,UTILITY,2,1,17,13,...,1033,6188,200,4721,9732,971,0,516,970,43


### Verifying DataFrame Columns for Snowflake Schema Creation
In this section, we check the columns of the DataFrame to ensure that we create the necessary columns in Snowflake. Based on the columns identified in the DataFrame, we need to create matching columns in Snowflake. This ensures that the structure of our Snowflake table aligns with the DataFrame's schema.

In [49]:
df_match.columns

Index(['puuid', 'riotIdGameName', 'win', 'lane', 'role', 'teamPosition',
       'kills', 'deaths', 'assists', 'totalMinionsKilled',
       'eligibleForProgression', 'timePlayed', 'killingSprees',
       'firstBloodKill', 'firstTowerKill', 'doubleKills', 'tripleKills',
       'quadraKills', 'pentaKills', 'wardsKilled', 'wardsPlaced',
       'visionWardsBoughtInGame', 'detectorWardsPlaced', 'objectivesStolen',
       'turretKills', 'dragonKills', 'enemyMissingPings', 'item0', 'item1',
       'item2', 'item3', 'item4', 'item5', 'item6', 'goldEarned',
       'largestKillingSpree', 'largestMultiKill',
       'magicDamageDealtToChampions', 'magicDamageTaken',
       'neutralMinionsKilled', 'participantId',
       'physicalDamageDealtToChampions', 'physicalDamageTaken', 'teamId',
       'totalDamageDealtToChampions', 'totalDamageTaken', 'totalHeal',
       'totalHealsOnTeammates', 'trueDamageDealtToChampions',
       'trueDamageTaken', 'visionScore'],
      dtype='object')

### Connecting to Snowflake and Verifying the Connection
In this section, we establish a connection to Snowflake using both the `snowflake.connector` and `SQLAlchemy` libraries. We also test the connection to ensure it is working correctly.

In [50]:
# We obtain the necessary credentials and connection details for Snowflake.
user = credentials.username
password = credentials.password
account = 'leygfyn-uob70869'
warehouse='COMPUTE_WH'
database='LEAGUE_OF_LEGENDS_DATA'
schema='LOL_MATCH_STATS'

#Connect to Snowflake using pandas to verify match_id
conn=snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    role='ACCOUNTADMIN'
)
# Create a cursor
cursor=conn.cursor()

# It's time to test this connection
cursor.execute("SELECT CURRENT_TIMESTAMP;")
result = cursor.fetchone()
print("Successful connection, timestamp snowflake:", result[0])

#Connect to Snowflake using sqlalchemy
connect_string = (
    f'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'
)
engine = create_engine(connect_string)


Successful connection, timestamp snowflake: 2024-07-25 12:56:21.042000-07:00


C:\Users\Styles\Desktop\KEVIN NO BORRE ESTO\DATA ENGINEER\Python_LeagueOfLeagues\venv\Lib\site-packages\snowflake\sqlalchemy\base.py:1068: SAWarning: The GenericFunction 'flatten' is already registered and is going to be overridden.
  functions.register_function("flatten", flatten)


### Inserting Match Data into Snowflake
In this section, we insert match data into a Snowflake table if it does not already exist. We handle potential errors and ensure that the data is properly inserted.

In [51]:
# We loop through each match ID in the `last_20_matches` list to process the match data.
for match in last_20_matches:
    match_id = match # match 
    table_name = "matches"
    query = f"SELECT COUNT(*) FROM {table_name} WHERE MATCH_ID='{match_id}'"
    cursor.execute(query)
    count = cursor.fetchone()
    # Check if Match Data Already Exists
    if count[0] == 0:
        # If the match data does not exist, we retrieve the match details from the API and process it into a DataFrame.
        endpoint = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
        res = requests.get(endpoint).json()
        df = pd.DataFrame(res['info']['participants'])
        df_match = df[stats].copy() # the purpose is do not create a view in order to use an independent copy
        # add match_id column to the match dataframe, so that I have even more possibilities in terms of queries
        df_match.loc[:, 'match_Id'] = res['metadata']['matchId']
        df_match.loc[:, 'gameMode'] = res['info']['gameMode']
        df_match.columns = df_match.columns.str.lower()
        # We insert the processed data into the Snowflake table using `SQLAlchemy`. If the insertion is successful, we print a success message
        try:
            with engine.connect() as connection:
                df_match.to_sql(table_name, connection, index=False, if_exists='append')
            print("success")
        except Exception as e:
            #if something goes wrong
            print(f'something has gone wrong {e}')
    # If the match data already exists in the table, we print a message indicating the number of existing records.
    else:
        print(f'it already exists {count[0]}')
# we close the Snowflake connection, cursor, and SQLAlchemy engine.
conn.close()
cursor.close()
engine.dispose()

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success


Export the match information (specifically the last one) **Optional**

In [63]:
df_match.to_csv(f'data/match_info{datetime.today().strftime("%Y_%m_%d")}.csv', index=False)

## **Extracting Item Information from an external League of Legends API**
In this section, we extract item information from and external League of Legends API by retrieving the current version and fetching the item data.

In [67]:
# Retrieve the Current API Version
version_lst = requests.get('http://ddragon.leagueoflegends.com/api/versions.json').json()
endpoint = f'http://ddragon.leagueoflegends.com/cdn/{version_lst[0]}/data/en_US/item.json'
res = requests.get(endpoint).json()
res['data']

{'1001': {'name': 'Boots',
  'description': '<mainText><stats><attention>25</attention> Move Speed</stats><br><br></mainText>',
  'colloq': ';',
  'plaintext': 'Slightly increases Move Speed',
  'into': ['3005',
   '3047',
   '3006',
   '3009',
   '3010',
   '3020',
   '3111',
   '3117',
   '3158'],
  'image': {'full': '1001.png',
   'sprite': 'item0.png',
   'group': 'item',
   'x': 0,
   'y': 0,
   'w': 48,
   'h': 48},
  'gold': {'base': 300, 'purchasable': True, 'total': 300, 'sell': 210},
  'tags': ['Boots'],
  'maps': {'11': True,
   '12': True,
   '21': True,
   '22': False,
   '30': False,
   '33': False},
  'stats': {'FlatMovementSpeedMod': 25}},
 '1004': {'name': 'Faerie Charm',
  'description': '<mainText><stats><attention>50%</attention> Base Mana Regen</stats><br><br></mainText>',
  'colloq': ';',
  'plaintext': 'Slightly increases Mana Regen',
  'into': ['3114', '4642', '3012'],
  'image': {'full': '1004.png',
   'sprite': 'item0.png',
   'group': 'item',
   'x': 48,
   '

### Extracting Item Information into a DataFrame
In this section, we extract item information from the JSON response and convert it into a pandas DataFrame for further analysis.

In [69]:
item_lst = []
# We loop through each item in the "res['data']" dictionary to extract relevant details such as "id", "name`, `plaintext`, `stats`, and gold values.
for iid, data in res['data'].items():
    item = {
        'id' : iid,
        'name': data['name'],
        'plaintext': data['plaintext'],
        'stats': data['stats'],
        'gold_base': data['gold']['base'],
        'gold_total': data['gold']['total'],
        'gold_sell': data['gold']['sell'],
    }
    item_lst.append(item)
df_items = pd.DataFrame(item_lst)
df_items

,id,name,plaintext,stats,gold_base,gold_total,gold_sell
0,1001,Boots,Slightly increases Move Speed,{'FlatMovementSpeedMod': 25},300,300,210
1,1004,Faerie Charm,Slightly increases Mana Regen,{},250,250,175
2,1006,Rejuvenation Bead,Slightly increases Health Regen,{},300,300,120
3,1011,Giant's Belt,Greatly increases Health,{'FlatHPPoolMod': 350},500,900,630
4,1018,Cloak of Agility,Increases critical strike chance,{'FlatCritChanceMod': 0.15},600,600,420
...,...,...,...,...,...,...,...
555,9404,Wandering Storms,,{},0,0,0
556,9405,Grizzly Smash,,{},0,0,0
557,9406,Lover's Ricochet,,{},0,0,0
558,9407,Hopped-Up Hex,,{},0,0,0


### Connecting to Snowflake and Inserting Item Data
In this section, we connect to Snowflake and insert item data into two separate tables: one for item details and another for item statistics. We use both `snowflake.connector` and `SQLAlchemy` for this process.

In [70]:
#Connect to Snowflake using pandas
conn=snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    role='ACCOUNTADMIN'
)
# Create a cursor
cursor=conn.cursor()

# It's time to test this connection
cursor.execute("SELECT CURRENT_TIMESTAMP;")
result = cursor.fetchone()
print("Successful connection, timestamp snowflake:", result[0])

#Connect to Snowflake using sqlalchemy
connect_string = (
    f'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'
)
engine = create_engine(connect_string)

# We prepare two DataFrames: "df_items" for item details and "df_items_stats" for item statistics. 
table_name_main = "items"
table_name_second = "items_stats"
# We normalize the "stats" column
df_items_stats = pd.json_normalize(df_items['stats'])
df_items = df_items.drop(columns=['stats'])
# Update column names to lowercase.
df_items.columns = df_items.columns.str.lower()
df_items_stats.columns = df_items_stats.columns.str.lower()
# We check if the main table "items" is empty. If it is, we insert the data from "df_items" and "df_items_stats" into their respective tables.
try:
    query = f"SELECT COUNT(*) FROM {table_name_main}"
    cursor.execute(query)
    count = cursor.fetchone()

    if count[0] == 0:
        with engine.connect() as connection:
            df_items.to_sql(table_name_main, connection, index=False, if_exists='append')
            df_items_stats.to_sql(table_name_second, connection, index=False, if_exists='append')
        print("success")
    # If the table already contains data, we print a message indicating that the table already exists.
    else:
        print("It exists")
except Exception as e:
    #if something goes wrong
    print(f'something has gone wrong {e}')

# We close the Snowflake connection, cursor, and SQLAlchemy engine.
conn.close()
cursor.close()
engine.dispose()

Successful connection, timestamp snowflake: 2024-07-24 17:24:29.619000-07:00
It exists


Export the items information into a csv **Optional**

In [71]:
df_items.to_csv(f'data/items_info{datetime.today().strftime("%Y_%m_%d")}.csv', index=False)

## **Extracting Champions Information from an external League of Legends API**
In this section, we extract champion information from and external League of Legends API by retrieving the current version and fetching the champion data.

In [78]:
# Retrieve the Current API Version
version_lst = requests.get('http://ddragon.leagueoflegends.com/api/versions.json').json()
endpoint = f'http://ddragon.leagueoflegends.com/cdn/{version_lst[0]}/data/en_US/champion.json'
res = requests.get(endpoint).json()

### Extracting Champion Information into a DataFrame
In this section, we extract champion information from the JSON response and convert it into a pandas DataFrame for further analysis.

In [79]:
champ_lst = []
for iid, data in res['data'].items():
    champ = {
        'id' : iid,
        'name': data['name'],
        'title': data['title'],
        'blurb': data['blurb'],
        'tags': data['tags'],
        'partype': data['partype'],
    }
    champ_lst.append(champ)
df_champs = pd.DataFrame(champ_lst)
df_champs

,id,name,title,blurb,tags,partype
0,Aatrox,Aatrox,the Darkin Blade,Once honored defenders of Shurima against the ...,[Fighter],Blood Well
1,Ahri,Ahri,the Nine-Tailed Fox,Innately connected to the magic of the spirit ...,"[Mage, Assassin]",Mana
2,Akali,Akali,the Rogue Assassin,Abandoning the Kinkou Order and her title of t...,[Assassin],Energy
3,Akshan,Akshan,the Rogue Sentinel,"Raising an eyebrow in the face of danger, Aksh...","[Marksman, Assassin]",Mana
4,Alistar,Alistar,the Minotaur,Always a mighty warrior with a fearsome reputa...,"[Tank, Support]",Mana
...,...,...,...,...,...,...
163,Zeri,Zeri,The Spark of Zaun,"A headstrong, spirited young woman from Zaun's...",[Marksman],Mana
164,Ziggs,Ziggs,the Hexplosives Expert,"With a love of big bombs and short fuses, the ...",[Mage],Mana
165,Zilean,Zilean,the Chronokeeper,"Once a powerful Icathian mage, Zilean became o...","[Support, Mage]",Mana
166,Zoe,Zoe,the Aspect of Twilight,"As the embodiment of mischief, imagination, an...",[Mage],Mana


### Connecting to Snowflake and Inserting Champion Data
In this section, we connect to Snowflake and insert champion data into a table. We use both `snowflake.connector` and `SQLAlchemy` for this process.

In [80]:
#Connect to Snowflake using pandas
conn=snowflake.connector.connect(
    user=user,
    password=password,
    account=account,
    warehouse=warehouse,
    database=database,
    schema=schema,
    role='ACCOUNTADMIN'
)
# Create a cursor
cursor=conn.cursor()

# It's time to test this connection
cursor.execute("SELECT CURRENT_TIMESTAMP;")
result = cursor.fetchone()
print("Successful connection, timestamp snowflake:", result[0])

#Connect to Snowflake using sqlalchemy
connect_string = (
    f'snowflake://{user}:{password}@{account}/{database}/{schema}?warehouse={warehouse}'
)
engine = create_engine(connect_string)
# We prepare the DataFrame "df_champs" for insertion by converting column names to lowercase and formatting the "tags" column.
table_name = "champions"
df_champs.columns = df_champs.columns.str.lower()
df_champs['tags'] = df_champs['tags'].apply(lambda x: ', '.join(x))
# We check if the "champions" table is empty. If it is, we insert the data from "df_champs" into the table. 
try:
    query = f"SELECT COUNT(*) FROM {table_name}"
    cursor.execute(query)
    count = cursor.fetchone()

    if count[0] == 0:
        with engine.connect() as connection:
            df_champs.to_sql(table_name, connection, index=False, if_exists='append')
        print("success")
    # If the table already contains data, we print a message indicating that the table already exists.
    else:
        print("It exists")
except Exception as e:
    #if something goes wrong
    print(f'something has gone wrong {e}')
# We close the Snowflake connection, cursor, and SQLAlchemy engine.
conn.close()
cursor.close()
engine.dispose()

Successful connection, timestamp snowflake: 2024-07-24 17:25:53.451000-07:00
It exists
